In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import math
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cityblock, hamming
from sklearn.preprocessing import normalize

In [ ]:
def check_numeric(X):
  newX = np.array(X).reshape(-1)
  return all(not isinstance(n, str) for n in newX)

In [ ]:
def error_return(p):
  ranked = np.arange(p, dtype=int) 
  weight = np.empty((1,p))
  weight = np.squeeze(weight) 
  weight[:] = np.nan
  return ranked, weight


### Preprocessing data and call feature selection algorithm

In [ ]:
def I_Relief2_configure(X,Y, **kwargs):
  X = np.array(X)
  Y = np.array(Y)
  numOfIterations, categoricalX, kernelWidth, theta = kwargs['numOfIterations'], kwargs['categoricalX'], kwargs['kernelWidth'], kwargs['theta']

  if not check_numeric(X):
    print('X does not contain numeric data')
    p = X.shape[1] # no of attributes
    return error_return(p) 

  # Check if the input sizes are consistent
  if Y.shape[0] != X.shape[0]:
    print('number of instances and output labels doesnot match')
    p = X.shape[1] # no of attributes
    return error_return(p)
  
  # converting classes as 0 to #classes
  [Y, grp] = pd.factorize(Y)

  # grpToInd contains class to index mapping, grpToInd[className]= ind
  grpToInd={}
  for ind, g in enumerate(grp):
    grpToInd[g]= ind

  # removing incomplete instances
  df_XY = pd.DataFrame(X)
  df_XY['Y'] = Y
  df_XY = df_XY.dropna()

  X, Y = np.array(df_XY.iloc[:, 0:-1]), np.array(df_XY.iloc[:, -1])
  Ngrp = len(grp)
  N = X.shape[0]
  C = np.zeros((N,Ngrp))
  C[np.arange(N), Y] = True

  
  # # Checking the number of classes
  # if Ngrp>2:
  #   print('Number of classes exceed 2')
  #   p = X.shape[1] # no of attributes
  #   return error_return(p)

  #  Do we have enough observations?
  if len(Y)<2:
    print('not enough instances')
    p = X.shape[1] # no of attributes
    return error_return(p)

  # Check number of iterations
  if (not check_numeric([numOfIterations]) or numOfIterations<=0):
    print('numOfIterations is invalid')
  else:
    numOfIterations = math.ceil(numOfIterations)
    
  # Check kernel width
  if (not check_numeric([kernelWidth]) or kernelWidth<=0):
    print('kernelWidth is invalid')

  # Check stop criterion theta
  if (not check_numeric([theta]) or theta<=0):
    print('Stop criterion theta is invalid')  

  # Check the type of categoricalX
  if not categoricalX or (categoricalX != 'on' and categoricalX != 'off'):
      print('categoricalX is invalid')
  categoricalX = (categoricalX == 'on')   

  # Find max and min for every predictor
  p = X.shape[1] # no of attributes
  Xmax = X.max(0)
  Xmin = X.min(0) 
  Xdiff = Xmax - Xmin
  Xmean = np.mean(X, axis=0) 

  # Exclude single-valued attributes
  isDiffValue = Xdiff >= 1e-9  # boolean array of size #attributes [1,0,0,0]
  if not any(isDiffValue):
    print("All attributes are single valued attributes.")
    p = X.shape[1] # no of attributes
    return error_return(p)
  
  X = X[:, isDiffValue ] 
  Xdiff = Xdiff[isDiffValue] 
  Xmean = Xmean[isDiffValue] 
  rejected = [ i for i in range(len(isDiffValue)) if not isDiffValue[i]]  # indices of the deleted attributes (values range from 1 to p)
  accepted = [ i for i in range(len(isDiffValue)) if isDiffValue[i]]  # indices of remaining attributes (values range from 1 to p)

  # Scale and center the attributes
  if not categoricalX:
      X = (X - Xmean) / Xdiff 

  # Call I_Relief2. By default all weights are set to NaN.
  weight = np.empty(p) 
  weight[:] = np.nan

  weight[accepted] = I_Relief2(X, Y, C, numOfIterations, categoricalX, kernelWidth, theta) 

  # Assign ranks to attributes
  sorted = np.argsort(-weight[accepted])
  accepted = np.array(accepted)
  ranked = accepted[sorted]
  ranked = np.append(ranked, rejected)
  ranked = ranked.astype(int)

  return ranked, weight



# I-Relief2 algorithm function

In [ ]:
def I_Relief2(scaledX, Y, C, numOfIterations, categoricalX, kernelWidth, theta):
  
  # I-Relief2 for classification
  numInstances,numAttr = scaledX.shape 
  attrWeights = np.ones((numAttr,1))/numAttr 
  # print(attrWeights)
  # C is boolean 2D matrix of size (N, Ngrp) i.e. (Number of instances vs Number of classes)
  numClasses = C.shape[1]   # Ngrp

   # Make searcher objects, one object per class. 
  instIdxPerClass = {}  
  for c in range(numClasses):
      c_C = C[:,c]
      instIdxPerClass[c] = np.array([i for i in range(len(c_C)) if c_C[i]], dtype=int)   # instances of class c
  
  # selecting distance function
  distFunc = cityblock
  distFunc2 = Cityblock
  if categoricalX:
    distFunc = hamming
    distFunc2 = Hamming
  else: 
    distFunc = cityblock
    distFunc2 = Cityblock
  
   # Outer loop, for updating attribute weights iteratively
  for i in range(numOfIterations):

    pairwiseWeightedDistances = np.array(pairwise_distances(scaledX, metric=distFunc, w=attrWeights))
    # print("pairwiseWeightedDistances : ", pairwiseWeightedDistances)
    pairwiseKernels = kernel_func(pairwiseWeightedDistances, kernelWidth)
    # print("pairwiseKernels : ", pairwiseKernels)
    pM = np.zeros((numInstances, numClasses, numInstances)) # probality of the i-th training example being nearest miss of the pattern x_n
    pH = np.zeros((numInstances, numInstances)) # probality of the i-th training example being nearest hit of the pattern x_n
    pO = np.zeros((numInstances, 1)) # probality of the i-th training example being outlier


    # calculating pM, pH, pO
    for j in range(numInstances):
      targetInstClass = Y[j]
      allMisses = np.array([], dtype=int)

      for c in range(numClasses):
        if c == targetInstClass: 
          hits = instIdxPerClass[c] # hit indices of class c with respect to jth target instance
          hits = hits[hits != j]
          hitKernels = pairwiseKernels[j, hits]
          pH[j, hits] = hitKernels/np.sum(hitKernels)
        else:
          misses = instIdxPerClass[c] # miss indices of class c with respect to jth target instance
          allMisses = np.append(allMisses, misses)
          missKernels = pairwiseKernels[j, misses]
          pM[j,c,misses] = missKernels/np.sum(missKernels)

      allMissKernels = pairwiseKernels[j, allMisses]
      allKernels = pairwiseKernels[j, :]
      pO[j,0] = sum(allMissKernels)/(sum(allKernels) - pairwiseKernels[j,j]) # deleting kernel of targer in the denominator 

    # print('pO', pO)
    # print('pM', pM)
    # print('pH', pH)


    # calculating v
    v = np.zeros((numAttr,1))

    for j in range(numInstances):
      targetInstClass = Y[j]
      targetInst = scaledX[j, :]
      m_bar = np.zeros((numAttr,1)) # shape: (numOfFeatures x 1)
      h_bar = np.zeros((numAttr,1)) # shape: (numOfFeatures x 1)
      m_bars = [] #list to hold all m_bar

      for c in range(numClasses):
        if c == targetInstClass: 
          hits = instIdxPerClass[c] # hit indices of class c with respect to jth target instance
          hits = hits[hits != j]
          hitInstances = scaledX[hits, :]
          h = np.transpose(distFunc2(targetInst, hitInstances)) # shape: (numOfFeatures x numOfHits)
          # print("H : " , h) 
          beta = pH[j, hits].reshape(-1,1) #shape: (numOfhits x 1) 
          h_bar = np.matmul(h, beta) # shape: (numOfFeatures x 1)
        else:
          misses = instIdxPerClass[c] # miss indices of class c with respect to jth target instance
          missInstances = scaledX[misses, :]
          m = np.transpose(distFunc2(targetInst, missInstances)) # shape: (numOfFeatures x numOfMisses) 
          # print("M : ", m)
          alpha = pM[j, c, misses].reshape(-1,1) #shape: (numOfMisses x 1)
          m_bar = np.matmul(m, alpha)            # shape: (numOfFeatures x 1)
          m_bars.append(m_bar)

      # getting the min(m_bar-h_bar)
      length = np.Inf
      min_mbar = np.zeros((numAttr,1))
      for m_bar in m_bars:
        l = np.linalg.norm(m_bar)
        if l<length:
          length =l
          min_mbar = m_bar
          
      gamma = (1 - pO[j])
      v = v +  gamma * (min_mbar - h_bar)
    
    v = v/numInstances
    # print("v= ", v)
    v[v<0] = 0 # taking vPlus
    # print("v+= ", v)
    newWeights = normalize(v, norm='l2', axis=0)
    # print("newWeights= ", newWeights)
    weightDiff = newWeights - attrWeights
    _, weightDiffNorm = normalize(weightDiff, norm='l2', axis=0, return_norm=True)
    attrWeights = newWeights

    if weightDiffNorm < theta:
      break

  return np.squeeze(attrWeights)    

In [ ]:
def kernel_func(d, kernelWidth):
  return np.exp(-d/kernelWidth)

In [ ]:
def Cityblock(thisX, X):
  d = abs(X - thisX) 
  return d

In [ ]:
def Hamming(thisX, X):
  d = (X != thisX).astype(int) 
  return d


In [ ]:
# def permute(x):
#   if len(x) == 7:
#     global X
#     global Y
#     Y.append((x[0] ^ x[1] ^ x[2]))
#     X.append(x)
#     return
#   x = np.append(x,0)
#   permute(x)
#   x = x[0:-1]
#   x = np.append(x,1)
#   permute(x)

In [ ]:
# X = []
# Y = []
# x = np.array([], dtype=int)
# permute(x)
# X = np.array(X)
# Y = np.array(Y)
# ranked, weight= I_Relief2_configure(X, Y, numOfIterations=5, categoricalX = 'on', kernelWidth = 3, theta = 1e-9)

# print('ranked')
# print(ranked)
# print('weight')
# print(weight)


In [ ]:
# # test classification dataset
# from sklearn.datasets import make_classification
# # define dataset
# X, Y = make_classification(n_samples=400, n_features=10, n_informative=5, n_redundant=0,shuffle=False, random_state=1)
# # summarize the dataset
# # X = X.astype(int)
# # print(X, Y)
# ranked, weight= I_Relief2_configure(X, Y, numOfIterations=10, categoricalX = 'off', kernelWidth = 3, theta = 1e-9)

# print('ranked')
# print(ranked)
# print('weight')
# print(weight)
# print('ranked weight')
# for r in ranked:
#   print(r,'th features weight= ',weight[r])